# BentoML Example: H2O Loan Default Prediction

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to __turn a H2O model into a docker image containing a REST API server__ serving this model, as well as distributing your model as a command line tool or a pip-installable PyPI package.

The notebook was built based on: https://github.com/kguruswamy/H2O3-Driverless-AI-Code-Examples/blob/master/Lending%20Club%20Data%20-%20H2O3%20Auto%20ML%20-%20Python%20Tutorial.ipynb

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=h2o&ea=h2o-loan-default-prediction&dt=h2o-loan-default-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install bentoml
!pip install h2o xlrd sklearn pandas numpy

In [2]:
import h2o
import bentoml
import numpy as np
import pandas as pd

import requests
import math
from sklearn import model_selection

h2o.init(strict_version_check=False)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "9.0.1"; Java(TM) SE Runtime Environment (build 9.0.1+11); Java HotSpot(TM) 64-Bit Server VM (build 9.0.1+11, mixed mode)
  Starting server from /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpx34yrz5f
  JVM stdout: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpx34yrz5f/h2o_bozhaoyu_started_from_python.out
  JVM stderr: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpx34yrz5f/h2o_bozhaoyu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,10 months and 7 days !!!
H2O cluster name:,H2O_from_python_bozhaoyu_7bamxr
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


## Prepare Dataset

In [3]:
%%bash

# Download training dataset
if [ ! -f ./LoanStats3c.csv.zip ]; then
    curl -O https://resources.lendingclub.com/LoanStats3c.csv.zip
fi

In [4]:
pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth',9999)
pd.set_option('display.max_columns',9999)
pd.set_option('display.max_rows',9999)

data_dictionary = pd.read_excel("https://resources.lendingclub.com/LCDataDictionary.xlsx")
data_dictionary

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
5,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
6,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


In [5]:
# Very first row has non-header data and hence skipping it. Read to a data frame
# Fix the Mon-Year on one column to be readable

def parse_dates(x):
    return datetime.strptime(x, "%b-%d")

lc = pd.read_csv("LoanStats3c.csv.zip", skiprows=1,verbose=False, parse_dates=['issue_d'],low_memory=False) 
lc.shape

(235631, 144)

In [6]:
lc.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Current', 'In Grace Period',
       'Late (31-120 days)', 'Default', 'Late (16-30 days)', nan],
      dtype=object)

In [7]:
# Keep just "Fully Paid" and "Charged Off" to make it a simple 'Yes' or 'No' - binary classification problem

lc = lc[lc.loan_status.isin(['Fully Paid','Charged Off'])]
lc.loan_status.unique()

array(['Fully Paid', 'Charged Off'], dtype=object)

In [8]:
# Drop the columns from the data frame that are Target Leakage ones
# Target Leakage columns are generally created in hindsight by analysts/data engineers/operations after an outcome 
# was detected in historical data. If we don't remove them now, they would climb to the top of the feature list after a model is built and 
# falsely increase the accuracy to 95% :) 
#
# In Production or real life scoring environment, don't expect these columns to be available at scoring time
# , that is,when someone applies for a loan. So we don't train on those columns ...

ignored_cols = [ 
                'out_prncp',                 # Remaining outstanding principal for total amount funded
                'out_prncp_inv',             # Remaining outstanding principal for portion of total amount 
                                             # funded by investors
                'total_pymnt',               # Payments received to date for total amount funded
                'total_pymnt_inv',           # Payments received to date for portion of total amount 
                                             # funded by investors
                'total_rec_prncp',           # Principal received to date 
                'total_rec_int',             # Interest received to date
                'total_rec_late_fee',        # Late fees received to date
                'recoveries',                # post charge off gross recovery
                'collection_recovery_fee',   # post charge off collection fee
                'last_pymnt_d',              # Last month payment was received
                'last_pymnt_amnt',           # Last total payment amount received
                'next_pymnt_d',              # Next scheduled payment date
                'last_credit_pull_d',        # The most recent month LC pulled credit for this loan
                'settlement_term',           # The number of months that the borrower will be on the settlement plan
                'settlement_date',           # The date that the borrower agrees to the settlement plan
                'settlement_amount',         # The loan amount that the borrower has agreed to settle for
                'settlement_percentage',     # The settlement amount as a percentage of the payoff balance amount on the loan
                'settlement_status',         # The status of the borrower’s settlement plan. Possible values are: 
                                             # COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAF
                'debt_settlement_flag',      # Flags whether or not the borrower, who has charged-off, is working with 
                                             # a debt-settlement company.
                'debt_settlement_flag_date'  # The most recent date that the Debt_Settlement_Flag has been set
                ]

lc = lc.drop(columns=ignored_cols, axis = 1)

In [9]:
# After dropping Target Leakage columns, we have 223K rows and 125 columns
lc.shape

(235543, 124)

In [10]:
import csv
import os 

train_path = os.getcwd() + "/train_lc.csv.zip"
test_path = os.getcwd() + "/test_lc.csv.zip"

train_lc, test_lc = model_selection.train_test_split(lc, test_size=0.2, random_state=10,stratify=lc['loan_status'])
train_lc.to_csv(train_path, index=False,compression="zip")
test_lc.to_csv(test_path, index=False,compression="zip")
print('Train LC shape', train_lc.shape)
print('Test LC shape', test_lc.shape)

# These two CSV files were created in the previous section
train_path = os.getcwd()+"/train_lc.csv.zip"
test_path = os.getcwd()+ "/test_lc.csv.zip"

train = h2o.load_dataset(train_path)
test = h2o.load_dataset(test_path)


train.describe()

Train LC shape (188434, 124)
Test LC shape (47109, 124)
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:188434
Cols:124




,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
type,int,int,int,int,int,enum,real,real,enum,enum,enum,enum,enum,real,enum,time,enum,enum,int,string,enum,enum,enum,enum,real,int,time,int,int,int,int,int,int,real,int,enum,int,int,int,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,enum,enum,enum,enum,int,real,time,time,time,int,int,enum,real,real,real
mins,NaN,NaN,1000.0,1000.0,950.0,,0.06,23.36,,,,,,3000.0,,1388534400000.0,,,NaN,NaN,,,,,0.0,0.0,-820540800000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,,0.0,0.0,1.0,,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,16.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,3.0,1.47,1485907200000.0,1491004800000.0,1485907200000.0,3.0,0.0,,4.41,174.15,0.04
mean,0.0,0.0,14884.780480168118,14884.780480168118,14879.984769203027,,0.13768163813324627,443.02308527123586,,,,,,74842.1021655859,,1403772635222.944,,,0.0,NaN,,,,,18.038431917806744,0.3439559739749727,878010488536.0388,0.7575596760669507,33.40950190769873,70.73781512605042,11.671577316195588,0.2224598533173423,16517.317517008567,0.5562211189754319,26.019354256662876,,0.015474914293598825,42.4452214452214,1.0,,0.0,0.0,0.0,0.0056200048823460734,280.02962841100884,139916.78260292622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30777.311727183012,0.0,0.0,0.0,4.403568358151916,13425.63404110787,8488.266061096774,64.59277420047215,0.010783616544784914,9.796506999798337,128.53483025519225,185.81975121262616,13.078101616481097,8.003645838861342,1.853216510820764,24.440105868864123,39.5963653177332,6.918806067907228,35.46866324059305,0.5053122048038038,3.686240275109585,5.803241453240905,4.646629589139977,8.54615939798552,8.57417451203075,8.277009456892086,15.304217922455612,5.767

In [11]:
import os

# These two CSV files were created in the previous section

train_path = os.getcwd()+"/train_lc.csv.zip"
test_path = os.getcwd()+ "/test_lc.csv.zip"

train = h2o.load_dataset(train_path)
test = h2o.load_dataset(test_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [12]:
train.describe()

Rows:188434
Cols:124




,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
type,int,int,int,int,int,enum,real,real,enum,enum,enum,enum,enum,real,enum,time,enum,enum,int,string,enum,enum,enum,enum,real,int,time,int,int,int,int,int,int,real,int,enum,int,int,int,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,enum,enum,enum,enum,int,real,time,time,time,int,int,enum,real,real,real
mins,NaN,NaN,1000.0,1000.0,950.0,,0.06,23.36,,,,,,3000.0,,1388534400000.0,,,NaN,NaN,,,,,0.0,0.0,-820540800000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,,0.0,0.0,1.0,,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,16.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,3.0,1.47,1485907200000.0,1491004800000.0,1485907200000.0,3.0,0.0,,4.41,174.15,0.04
mean,0.0,0.0,14884.780480168118,14884.780480168118,14879.984769203027,,0.13768163813324627,443.02308527123586,,,,,,74842.1021655859,,1403772635222.944,,,0.0,NaN,,,,,18.038431917806744,0.3439559739749727,878010488536.0388,0.7575596760669507,33.40950190769873,70.73781512605042,11.671577316195588,0.2224598533173423,16517.317517008567,0.5562211189754319,26.019354256662876,,0.015474914293598825,42.4452214452214,1.0,,0.0,0.0,0.0,0.0056200048823460734,280.02962841100884,139916.78260292622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30777.311727183012,0.0,0.0,0.0,4.403568358151916,13425.63404110787,8488.266061096774,64.59277420047215,0.010783616544784914,9.796506999798337,128.53483025519225,185.81975121262616,13.078101616481097,8.003645838861342,1.853216510820764,24.440105868864123,39.5963653177332,6.918806067907228,35.46866324059305,0.5053122048038038,3.686240275109585,5.803241453240905,4.646629589139977,8.54615939798552,8.57417451203075,8.277009456892086,15.304217922455612,5.767

## Model Training

In [21]:
from h2o.automl import H2OAutoML

# Identify predictors and response
x = train.columns
y = "loan_status"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML 
aml = H2OAutoML(project_name='LP', 
                max_models=1,         # 1 base models *FOR DEMO PURPOSE
                balance_classes=True,  # Doing smart Class imbalance sampling
                max_runtime_secs=3600,  # 1 hour *FOR DEMO PURPOSE (need to be longer runtime or else model will not train)
                seed=1234)             # Set a seed for reproducability
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████| 100%


### View the AutoML Leaderboard

In [23]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_20200224_151844,0.705681,0.4271,0.49588,0.366057,0.133997


In [24]:
test_pc = aml.predict(test)
test_pc

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,Charged Off,Fully Paid
Fully Paid,0.0300723,0.969928
Fully Paid,0.038023,0.961977
Fully Paid,0.0306047,0.969395
Fully Paid,0.242732,0.757268
Fully Paid,0.0589932,0.941007
Fully Paid,0.133116,0.866884
Fully Paid,0.34577,0.65423
Charged Off,0.432134,0.567866
Fully Paid,0.379319,0.620681
Fully Paid,0.0814494,0.918551


## Define BentoService for model serving

In [25]:
%%writefile loan_prediction.py

import h2o

from bentoml import api, env, artifacts, BentoService
from bentoml.artifact import H2oModelArtifact
from bentoml.adapters import DataframeInput

@env(
    pip_dependencies=['h2o==3.24.0.2', 'pandas'],
    conda_channels=['h2oai'],
    conda_dependencies=['h2o==3.24.0.2']
)
@artifacts([H2oModelArtifact('model')])
class LoanPrediction(BentoService):
    
    @api(input=DataframeInput())
    def predict(self, df):
        h2o_frame = h2o.H2OFrame(df, na_strings=['NaN'])
        predictions = self.artifacts.model.predict(h2o_frame)
        return predictions.as_data_frame()

Overwriting loan_prediction.py


## Save BentoService to file archive

In [26]:
# 1) import the custom BentoService defined above
from loan_prediction import LoanPrediction

# 2) `pack` it with required artifacts
bentoml_svc = LoanPrediction()
bentoml_svc.pack('model', aml.leader)

# 3) save your BentoSerivce
saved_path = bentoml_svc.save()

[2020-02-24 15:39:34,382] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-24 15:39:35,026] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'
no previously-included directories found matching 'scripts'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.6.2+16.g7795c2f
creating BentoML-0.6.2+16.g7795c2f/BentoML.egg-info
creating BentoML-0.6.2+16.g7795c2f/bentoml
creating BentoML-0.6.2+16.g7795c2f/bentoml/artifact
creating BentoML-0.6.2+16.g7795c2f/bentoml/bundler
creating BentoML-0.6.2+16.g7795c2f/bentoml/cli
creating BentoML-0.6.2+16.g7795c2f/bentoml/clipper
creating BentoML-0.6.2+16.g7795c2f/bentoml/configuration
creating BentoML-0.6.2+16.g7795c2f/bentoml/deployment
creating BentoML-0.6.2+16.g7795c2f/bentoml/deployment/aws_lambda
creating BentoML-0.6.2+16.g7795c2f/bentoml/deployment/sagemaker
creating BentoML-0.6.2+16.g7795c2f/bentoml/handlers
creating BentoML-0.6.2+16.g7795c2f/bentoml/marshal
creating BentoML-0.6.2+16.g7795c2f/bentoml/migrations
creating BentoML-0.6.2+16.g7795c2f/bentoml/migrations/versions
creating BentoML-0.6.2+16.g7795c2f/bentoml/proto
creating BentoML-0.6.2+16.g7795c2f/bentoml/repository
creating BentoML-0.6.2+16.g7795c2f/bentoml/server
creating BentoML-0.6.2+16.g7795c2f/bentoml/server/static

copying bentoml/proto/status_pb2.py -> BentoML-0.6.2+16.g7795c2f/bentoml/proto
copying bentoml/proto/yatai_service_pb2.py -> BentoML-0.6.2+16.g7795c2f/bentoml/proto
copying bentoml/proto/yatai_service_pb2_grpc.py -> BentoML-0.6.2+16.g7795c2f/bentoml/proto
copying bentoml/repository/__init__.py -> BentoML-0.6.2+16.g7795c2f/bentoml/repository
copying bentoml/repository/metadata_store.py -> BentoML-0.6.2+16.g7795c2f/bentoml/repository
copying bentoml/server/__init__.py -> BentoML-0.6.2+16.g7795c2f/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.6.2+16.g7795c2f/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.6.2+16.g7795c2f/bentoml/server
copying bentoml/server/gunicorn_config.py -> BentoML-0.6.2+16.g7795c2f/bentoml/server
copying bentoml/server/gunicorn_server.py -> BentoML-0.6.2+16.g7795c2f/bentoml/server
copying bentoml/server/marshal_server.py -> BentoML-0.6.2+16.g7795c2f/bentoml/server
copying bentoml/server/middlewares.py -> BentoML

## Load BentoService from archive

In [125]:
import pandas as pd

loaded_bentoml_svc = bentoml.load(saved_path)
sample_data = pd.read_csv('sample_data.csv')
result = loaded_bentoml_svc.predict(sample_data)
print(result)

[2020-02-24 17:21:25,109] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-24 17:21:25,121] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+16.g7795c2f
[2020-02-24 17:21:25,122] WARNING - Module `loan_prediction` already loaded, using existing imported module.
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 hours 30 mins
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,10 months and 7 days !!!
H2O cluster name:,H2O_from_python_bozhaoyu_7bamxr
H2O cluster total nodes:,1
H2O cluster free memory:,3.805 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


[2020-02-24 17:21:25,414] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
       predict  Charged Off  Fully Paid
0  Charged Off     0.436739    0.563261
1   Fully Paid     0.056414    0.943586


## Use BentoService with BentoML CLI

`bentoml get` list all versions of BentoService, when version information is not provided

In [128]:
!bentoml get LoanPrediction

BENTO_SERVICE                         AGE                    APIS                       ARTIFACTS
LoanPrediction:20200224153935_977ED8  1 hour and 53 minutes  predict<DataframeHandler>  model<H2oModelArtifact>


With `version` tag, `bentoml get` will display additional information and metadata

In [31]:
!bentoml get LoanPrediction:20200224153935_977ED8

{
  "name": "LoanPrediction",
  "version": "20200224153935_977ED8",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/LoanPrediction/20200224153935_977ED8"
  },
  "bentoServiceMetadata": {
    "name": "LoanPrediction",
    "version": "20200224153935_977ED8",
    "createdAt": "2020-02-24T23:39:51.835472Z",
    "env": {
      "condaEnv": "name: bentoml-LoanPrediction\nchannels:\n- defaults\n- h2oai\ndependencies:\n- python=3.7.3\n- pip\n- h2o==3.24.0.2\n",
      "pipDependencies": "bentoml==0.6.2\nh2o==3.24.0.2\npandas",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "H2oModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "orient": "records",
          "typ": "frame",
          "input_dtypes": null,
          "output_orient": "records"
        }
      }

Get prediction result from command line with `bentoml run` command

In [127]:
!bentoml run LoanPrediction:latest predict --input sample_data.csv

[2020-02-24 17:30:05,013] INFO - Getting latest version LoanPrediction:20200224153935_977ED8
[2020-02-24 17:30:05,014] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-24 17:30:05,028] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+16.g7795c2f
[2020-02-24 17:30:05,114] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundl



## Model Serving via REST API

In your termnial, run the following command to start the REST API server:

In [139]:
!bentoml serve {saved_path}

[2020-02-25 11:18:28,286] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-25 11:18:28,301] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+19.ge8fc860
[2020-02-25 11:18:28,381] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will b

Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.

# Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this demo, we are going to deploy to AWS Sagemaker

In [134]:
!bentoml sagemaker deploy h2o-loan-prediction -b LoanPrediction:20200224153935_977ED8 --api-name predict

Deploying Sagemaker deployment |[2020-02-25 10:45:28,356] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-02-25 10:45:28,356] INFO - 

[2020-02-25 10:45:28,356] INFO -  ---> 406f2b43ea59

[2020-02-25 10:45:28,356] INFO - Step 2/11 : EXPOSE 8080
[2020-02-25 10:45:28,356] INFO - 

[2020-02-25 10:45:28,357] INFO -  ---> Using cache

[2020-02-25 10:45:28,357] INFO -  ---> 58636f0540f4

[2020-02-25 10:45:28,357] INFO - Step 3/11 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential     && apt-get install -y nginx      && rm -rf /var/lib/apt/lists/*
[2020-02-25 10:45:28,357] INFO - 

[2020-02-25 10:45:28,357] INFO -  ---> Using cache

[2020-02-25 10:45:28,357] INFO -  ---> 70d334258584

[2020-02-25 10:45:28,357] INFO - Step 4/11 : RUN conda install pip numpy scipy       && pip install gunicorn gevent
[2020-02-25 10:45:28,357] INFO - 

[2020-02-25 10:45:28,357] INFO -  ---> Using cache

[2020-02-25 10

openjdk-8.0.152      | 57.9 MB   | #########4 |  94% 
openjdk-8.0.152      | 57.9 MB   | #########4 |  95% 
openjdk-8.0.152      | 57.9 MB   | #########5 |  96% 
openjdk-8.0.152      | 57.9 MB   | #########6 |  97% 
openjdk-8.0.152      | 57.9 MB   | #########7 |  97% 
openjdk-8.0.152      | 57.9 MB   | #########8 |  98% 
openjdk-8.0.152      | 57.9 MB   | #########9 | 100% 
openjdk-8.0.152      | 57.9 MB   | ########## | 100% 
[2020-02-25 10:46:01,261] INFO - 
python-3.7.3         | 32.1 MB   |            |   0% 
python-3.7.3         | 32.1 MB   |            |   0% 
python-3.7.3         | 32.1 MB   | 1          |   1% 
python-3.7.3         | 32.1 MB   | 1          |   2% 
python-3.7.3         | 32.1 MB   | 3          |   3% 
python-3.7.3         | 32.1 MB   | 5          |   5% 
python-3.7.3         | 32.1 MB   | 6          |   6% 
python-3.7.3         | 32.1 MB   | 6          |   7% 
python-3.7.3         | 32.1 MB   | 7          |   8% 
python-3.7.3         | 32.1 MB   | 8          | 

h2o-3.18.0.2         | 118.2 MB  | ###6       |  36% 
h2o-3.18.0.2         | 118.2 MB  | ###6       |  37% 
h2o-3.18.0.2         | 118.2 MB  | ###7       |  37% 
h2o-3.18.0.2         | 118.2 MB  | ###7       |  38% 
h2o-3.18.0.2         | 118.2 MB  | ###8       |  38% 
h2o-3.18.0.2         | 118.2 MB  | ###8       |  39% 
h2o-3.18.0.2         | 118.2 MB  | ###9       |  39% 
h2o-3.18.0.2         | 118.2 MB  | ####       |  40% 
h2o-3.18.0.2         | 118.2 MB  | ####1      |  41% 
h2o-3.18.0.2         | 118.2 MB  | ####1      |  42% 
h2o-3.18.0.2         | 118.2 MB  | ####2      |  43% 
h2o-3.18.0.2         | 118.2 MB  | ####3      |  43% 
h2o-3.18.0.2         | 118.2 MB  | ####3      |  44% 
h2o-3.18.0.2         | 118.2 MB  | ####4      |  45% 
h2o-3.18.0.2         | 118.2 MB  | ####5      |  45% 
h2o-3.18.0.2         | 118.2 MB  | ####6      |  46% 
h2o-3.18.0.2         | 118.2 MB  | ####7      |  47% 
h2o-3.18.0.2         | 118.2 MB  | ####7      |  48% 
h2o-3.18.0.2         | 118.2

\[2020-02-25 10:47:38,142] INFO - Collecting ruamel.yaml>=0.15.0

[2020-02-25 10:47:38,151] INFO -   Downloading ruamel.yaml-0.16.10-py2.py3-none-any.whl (111 kB)

-[2020-02-25 10:47:38,278] INFO - Collecting cerberus

/[2020-02-25 10:47:38,290] INFO -   Downloading Cerberus-1.3.2.tar.gz (52 kB)

-[2020-02-25 10:47:38,606] INFO - Collecting python-json-logger

[2020-02-25 10:47:38,615] INFO -   Downloading python-json-logger-0.1.11.tar.gz (6.0 kB)

|[2020-02-25 10:47:38,854] INFO - Requirement already satisfied: numpy in /opt/conda/lib/python3.7/site-packages (from bentoml==0.6.2->-r /opt/program/requirements.txt (line 1)) (1.18.1)

-[2020-02-25 10:47:39,044] INFO - Collecting protobuf>=3.6.0

[2020-02-25 10:47:39,057] INFO -   Downloading protobuf-3.11.3-cp37-cp37m-manylinux1_x86_64.whl (1.3 MB)

\[2020-02-25 10:47:40,571] INFO - Collecting grpcio

[2020-02-25 10:47:40,581] INFO -   Downloading grpcio-1.27.2-cp37-cp37m-manylinux2010_x86_64.whl (2.7 MB)

|[2020-02-25 10:47:41,7

|[2020-02-25 10:48:03,837] INFO -   Building wheel for alembic (setup.py): finished with status 'done'

[2020-02-25 10:48:03,839] INFO -   Created wheel for alembic: filename=alembic-1.4.0-py2.py3-none-any.whl size=157563 sha256=6eed3407447e5a1b56c8400ff6018f272b82ae4d70731ed3655ea5a47477136f

[2020-02-25 10:48:03,839] INFO -   Stored in directory: /root/.cache/pip/wheels/33/a9/f9/a53f885636269db5b76cf7afa3a1ab86d9d2fe96610d09274e

[2020-02-25 10:48:03,841] INFO -   Building wheel for tabulate (setup.py): started

/[2020-02-25 10:48:04,112] INFO -   Building wheel for tabulate (setup.py): finished with status 'done'

[2020-02-25 10:48:04,113] INFO -   Created wheel for tabulate: filename=tabulate-0.8.6-py3-none-any.whl size=23273 sha256=d08b9af63dc777d7cb1d20e88a69b0977a951750d6eec812dc38333d0d942ad2

[2020-02-25 10:48:04,113] INFO -   Stored in directory: /root/.cache/pip/wheels/09/b6/7e/08b4ee715a1239453e89a59081f0ac369a9036f232e013ecd8

[2020-02-25 10:48:04,115] INFO -   Building 

In [135]:
!bentoml sagemaker get h2o-loan-prediction

{
  "namespace": "bobo",
  "name": "h2o-loan-prediction",
  "spec": {
    "bentoName": "LoanPrediction",
    "bentoVersion": "20200224153935_977ED8",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict"
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "bobo-h2o-loan-prediction",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-h2o-loan-prediction",
      "EndpointConfigName": "bobo-h2o-loan-prediction-LoanPrediction-20200224153935-977ED8",
      "ProductionVariants": [
        {
          "VariantName": "bobo-h2o-loan-prediction-LoanPrediction-20200224153935-977ED8",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/loanprediction-sagemaker:20200224153935_977ED8",
              "ResolvedImage": "192023623294.dkr.ecr.us

In [137]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name bobo-h2o-loan-prediction \
--body fileb:///Users/bozhaoyu/src/bento_gallery/h2o/loan-prediction/sample_data.csv \
--content-type text/csv output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "bobo-h2o-loan-prediction-LoanPrediction-20200224153935-977ED8"
}
[{"predict":"Charged Off","Charged Off":0.4367393851,"Fully Paid":0.5632606149},{"predict":"Fully Paid","Charged Off":0.0564144254,"Fully Paid":0.9435855746}]

In [138]:
!bentoml sagemaker delete h2o-loan-prediction

Successfully deleted AWS Sagemaker deployment "h2o-loan-prediction"
